In [1]:
import tensorflow as tf

x = tf.Variable(3,name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [3]:
# with block
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [4]:
init = tf.global_variables_initializer() # prepare an init node

with tf.Session() as sess:
    init.run() # actually init all variables
    result = f.eval()
    print(result)

42


In [5]:
# Using interactive session
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [6]:
# Nodes created are automatically added to default graph
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [7]:
# Using a different graph
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [8]:
x2.graph is tf.get_default_graph()

False

In [9]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

# This evaluates x twice
with tf.Session() as sess:
    print(y.eval()) # should be 10
    print(z.eval()) # should be 15

10
15


In [10]:
# This evaluates x once
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [11]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="x")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [12]:
housing

{'DESCR': 'California housing dataset.\n\nThe original database is available from StatLib\n\n    http://lib.stat.cmu.edu/datasets/\n\nThe data contains 20,640 observations on 9 variables.\n\nThis dataset contains the average house value as target variable\nand the following input variables (features): average income,\nhousing average age, average rooms, average bedrooms, population,\naverage occupation, latitude, and longitude in that order.\n\nReferences\n----------\n\nPace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,\nStatistics and Probability Letters, 33 (1997) 291-297.\n\n',
 'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.     

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)),scaled_housing_data]

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
#X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

##### Manually computing the gradients
# gradients = 2/m * tf.matmul(tf.transpose(X), error)

##### Using autodiff
#gradients = tf.gradients(mse, [theta])[0]
#training_op = tf.assign(theta, theta - learning_rate * gradients)

##### Using an optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [14]:
scaled_housing_data_plus_bias[:10]

array([[ 1.        ,  2.34476576,  0.98214266,  0.62855945, -0.15375759,
        -0.9744286 , -0.04959654,  1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891,  0.32704136, -0.26333577,
         0.86143887, -0.09251223,  1.04318455, -1.32284391],
       [ 1.        ,  1.7826994 ,  1.85618152,  1.15562047, -0.04901636,
        -0.82077735, -0.02584253,  1.03850269, -1.33282653],
       [ 1.        ,  0.93296751,  1.85618152,  0.15696608, -0.04983292,
        -0.76602806, -0.0503293 ,  1.03850269, -1.33781784],
       [ 1.        , -0.012881  ,  1.85618152,  0.3447108 , -0.03290586,
        -0.75984669, -0.08561576,  1.03850269, -1.33781784],
       [ 1.        ,  0.08744664,  1.85618152, -0.26972966,  0.01466934,
        -0.89407076, -0.08961842,  1.03850269, -1.33781784],
       [ 1.        , -0.11136631,  1.85618152, -0.2009177 , -0.3066332 ,
        -0.29271158, -0.0907249 ,  1.03382082, -1.33781784],
       [ 1.        , -0.39513665,  1.85618152, -0.25523193, -0

In [15]:
scaled_housing_data_plus_bias[1:3]

array([[ 1.        ,  2.33223796, -0.60701891,  0.32704136, -0.26333577,
         0.86143887, -0.09251223,  1.04318455, -1.32284391],
       [ 1.        ,  1.7826994 ,  1.85618152,  1.15562047, -0.04901636,
        -0.82077735, -0.02584253,  1.03850269, -1.33282653]])

In [16]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    

Epoch 0 MSE = 11.611621
Epoch 100 MSE = 0.79563856
Epoch 200 MSE = 0.5925095
Epoch 300 MSE = 0.56937313
Epoch 400 MSE = 0.5573703
Epoch 500 MSE = 0.5488461
Epoch 600 MSE = 0.5425982
Epoch 700 MSE = 0.5379947
Epoch 800 MSE = 0.5345921
Epoch 900 MSE = 0.532069


In [17]:
##### Placeholders

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
print(B_val_1)

[[6. 7. 8.]]


In [18]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [19]:
##### Mini-batch Gradient Descent + model saving

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

saver = tf.train.Saver()

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
            print("Epoch", epoch, "MSE =", mse.eval())
        for batch_start in range(0, n_batches, batch_size):
            batch_stop = min(batch_start+batch_size, m)
            X_batch = scaled_housing_data_plus_bias[batch_start:batch_stop]
            y_batch = housing.target[batch_start:batch_stop].reshape(-1,1)
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        
    best_theta = theta.eval
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 10.753413
Epoch 100 MSE = 0.5428468
Epoch 200 MSE = 0.5328625
Epoch 300 MSE = 0.5284619
Epoch 400 MSE = 0.5264162
Epoch 500 MSE = 0.52542037
Epoch 600 MSE = 0.52491504
Epoch 700 MSE = 0.52464867
Epoch 800 MSE = 0.5245041
Epoch 900 MSE = 0.5244247


In [20]:
##### Generate log for tensorboard

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        for batch_start in range(0, n_batches, batch_size):
            if batch_start % (batch_size * 10) == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + (batch_start/batch_size)
                file_writer.add_summary(summary_str, step)
            batch_stop = min(batch_start+batch_size, m)
            X_batch = scaled_housing_data_plus_bias[batch_start:batch_stop]
            y_batch = housing.target[batch_start:batch_stop].reshape(-1,1)
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
file_writer.close()

Epoch 0 MSE = 11.451897
Epoch 100 MSE = 0.6168668
Epoch 200 MSE = 0.5603736
Epoch 300 MSE = 0.5386234
Epoch 400 MSE = 0.53013587
Epoch 500 MSE = 0.5267614
Epoch 600 MSE = 0.52538615
Epoch 700 MSE = 0.52480745
Epoch 800 MSE = 0.52455306
Epoch 900 MSE = 0.52443737


In [21]:
##### Variable names will get suffixes if they already exist.

logdir = "{}/scope-{}/".format(root_logdir, now)
tf.reset_default_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")
#with tf.Session() as sess:
#    sess.run(output, feed_dict={X: [[1., 2.],[4.,5.],[7.,8.]]})

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
file_writer.close()

In [22]:
##### Name scopes
logdir = "{}/scoperelu-{}/".format(root_logdir, now)
tf.reset_default_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]),1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="relu")
    
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")


file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
file_writer.close()

In [24]:
##### Sharing variable
logdir = "{}/sharevar-{}/".format(root_logdir, now)
tf.reset_default_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold") # reuse existing variable
        w_shape = (int(X.get_shape()[1]),1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")
    
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"): # create the variable
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")


file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
file_writer.close()